In [1]:
import numpy as np
import pandas as pd
import string
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
import matplotlib
import langid
import re

from matplotlib import pyplot as plt
# import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
dataFull = pd.read_csv("../Instagram_Like_Predictor/InstagramDatasetFinalVersion.csv",index_col = 0)
dataFull.head()
dataText = dataFull.loc[:,['hashtags','mentions','location','edge_media_to_caption']]
dataLabels = dataFull.loc[:,['mean_likes']]

dataText['mentions'].replace('[]','',inplace=True)
dataText['hashtags'].replace('[]', '', inplace=True)
#fill nan w str
dataText.fillna({'location':''},inplace=True)
dataText.fillna({'edge_media_to_caption':''},inplace = True)
# convert to str
dataText['edge_media_to_caption']= dataText['edge_media_to_caption'].astype(str)
dataText['location'] = dataText['location'].astype(str)
dataText['mentions'] = dataText['mentions'].astype(str)
dataText['hashtags'] = dataText['hashtags'].astype(str)

dataText.head()
#dataText.to_csv('NLP.csv', index=None, header= True)
dataText.info()



<class 'pandas.core.frame.DataFrame'>

📧takashi.yasui1980@gmail.com
Data columns (total 4 columns):
hashtags                 20668 non-null object
mentions                 20668 non-null object
location                 20668 non-null object
edge_media_to_caption    20668 non-null object
dtypes: object(4)
memory usage: 807.3+ KB


In [3]:
# bag of words, tfidf, autoencoder, 
def remove_punctuation(text):
    '''a function for removing punctuation'''
    import string
    # replacing the punctuations with no space, 
    # which in effect deletes the punctuation marks 
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)

dataText['edge_media_to_caption'] = dataText['edge_media_to_caption'].apply(remove_punctuation)
dataText['location'] = dataText['location'].apply(remove_punctuation)
dataText['mentions'] = dataText['mentions'].apply(remove_punctuation)
dataText['hashtags'] = dataText['hashtags'].apply(remove_punctuation)
dataText.head()
#dataText.to_csv('NLP.csv',index = None, header=True)


,hashtags,mentions,location,edge_media_to_caption
biography,,,,
NaN,,,,OCLOCK이 발매되었습니다 여러분들의 많은 관심과 사랑부탁드릴께요❤️
NaN,,,,즐거운 화이트데이메이트데이🧛‍♂️❤️
NaN,,,,OCLOCK 곧 12시죠 두구두구 티저 이미지 공개 😆
NaN,,,,None
NaN,,,,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요


In [4]:
# Remove Emojis id lan
dataText["edge_media_to_caption"] = dataText['edge_media_to_caption'].str.replace('[^\w\s]','') #remove emojis

def identify_language(row):
    lang = langid.classify(row['edge_media_to_caption'])
    return lang[0]
#depends on if langid wants to work
#dataText['language'] = dataText.apply(identify_language,axis=1)
dataText.head(20)

,hashtags,mentions,location,edge_media_to_caption,language
biography,,,,,
NaN,,,,OCLOCK이 발매되었습니다 여러분들의 많은 관심과 사랑부탁드릴께요,ko
NaN,,,,즐거운 화이트데이메이트데이,ko
NaN,,,,OCLOCK 곧 12시죠 두구두구 티저 이미지 공개,ko
NaN,,,,None,en
NaN,,,,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요,ko
NaN,,,,빼꼼 메이 사랑해요,ko
NaN,,,,복 많이받으세요,ko
NaN,,,,이젠 어렵지 않아요,ko
NaN,,,,다음번에는 예고없이 바로 방송킬꺼에요,ko


In [5]:
# bag of words, tfidf, autoencoder, 
#everything lower case and removed stopwords
from nltk.corpus import stopwords
sw = stopwords.words('english')
np.array(sw)
def stopwords(text):
    text = [word.lower()for word in text.split() if word.lower() not in sw]
    return " ".join(text)
dataText['edge_media_to_caption'] = dataText['edge_media_to_caption'].apply(stopwords)

def lower(text):
    text = [word.lower() for word in text.split()]
    return " ".join(text)

dataText['location']= dataText['location'].apply(lower)
dataText['hashtags']= dataText['hashtags'].apply(lower)
dataText['mentions'] = dataText['mentions'].apply(lower)
dataText.head()
#dataText.to_csv('NLP.csv',index = None, header=True)




In [6]:
#Translate then stem words then apply vectorizer

from googletrans import Translator
translator = Translator()
def translate(row):
    translator = Translator()
    if str(row['language']) =='en':
        return str(row['edge_media_to_caption'])
    else:
        try:
            english = translator.translate(str(row['edge_media_to_caption']),src = row['language'],dest = 'en').text
            return english
        except:
            return ''
#dataText['english'] = dataText.apply(translate,axis=1)

dataText.head()

,hashtags,mentions,location,edge_media_to_caption,language,english
biography,,,,,,
NaN,,,,oclock이 발매되었습니다 여러분들의 많은 관심과 사랑부탁드릴께요,ko,oclock has been released.
NaN,,,,즐거운 화이트데이메이트데이,ko,Happy White Day Mate Day
NaN,,,,oclock 곧 12시죠 두구두구 티저 이미지 공개,ko,oclock shortly 12 o'clock
NaN,,,,none,en,none
NaN,,,,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요,ko,Today is Valentine's Day.


In [7]:
#dataText.to_csv('NLP.csv',index = None, header=True)

In [15]:
dataText = pd.read_csv("NLP.csv", encoding = "ISO-8859-1")

In [19]:

dataText.fillna({'english':''},inplace = True)
dataText.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20668 entries, 0 to 20667
Data columns (total 6 columns):
hashtags                 8746 non-null object
mentions                 9180 non-null object
location                 6764 non-null object
edge_media_to_caption    20017 non-null object
language                 20668 non-null object
english                  20668 non-null object
dtypes: object(6)
memory usage: 968.9+ KB


In [21]:
dataText['english'] = dataText['english'].apply(remove_punctuation)
dataText['english'] = dataText['english'].apply(stopwords)

In [22]:
dataText.head()

,hashtags,mentions,location,edge_media_to_caption,language,english
0,NaN,NaN,NaN,oclock? ??????? ????? ?? ??? ????????,ko,oclock released
1,NaN,NaN,NaN,??? ??????????,ko,happy white day mate day
2,NaN,NaN,NaN,oclock ? 12?? ???? ?? ??? ??,ko,oclock shortly 12 oclock
3,NaN,NaN,NaN,none,en,none
4,NaN,NaN,NaN,?? ???????? ????? ? ???????,ko,today valentines day


In [29]:
## first tokenize, then stemmer, then vectorize into number
stemmer = SnowballStemmer('english')
def tokenize_stem(row):
    stem_result = []
    #input_str = str(row['english'])
    input_str = row
    input_str = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", input_str)
    tokens = word_tokenize(input_str)
    for word in tokens:
        stem_result.append(stemmer.stem(word))
    str1 = ' '.join(stem_result)
    return str1
dataText['tokenStem'] = dataText['english'].apply(tokenize_stem)  
    

oclock releas
happi white day mate day
oclock short oclock
none
today valentin day
love
happi new year
difficult
next time well air without notic
see soon
max cute mani cushion
add promo rexonaarg particip trip manchest train wait canchasign new surefootbal contest come train manchest see field uk argentina movementiseveryth manciti
hack game unlock new level puma pumafootbal
manchesterc manciti premierleagu manchest
rip bernard halford true legend club legend
leomessi thank invit call diainternacionaldelsindromededown sumo nomino friend maluma nicolasotamendi30 pocho22lavezzidiferentesperoigual
fue lindo vert otra vez camiseta prometida great see promis shirt

mom love
manchesterc manciti manchest premierleagu puma

manchesterc manciti carabaocup champion campeon
make hallway fight devindruid
forev think everyth

get beauti sleep tbt

find good spar partner
hair check
moment monet tbt
claycept tbt
fam tbt rossbutl
season



canyaman erkencikus super depart
brush sub femal player labor

choos love time choos kind timeagnezmo
35mm rehears agnezmo
late post agnezmo
agnezmo
agnezmo
backstageagnezmo
throwback miss bali balican pls go back crazybonz
socal vibe
pretti cafebirdiela las highland park neighborhood delici littl produceforward plate lot tasti pasta bounti cocktail way like
good morn palm spring
sunset coachellath best time day swipe throughalso love today shirt handdrawn illustr friend unnecessaryobserv samortizphoto
coachilllllin fave act far janellemona slay the1975 christineandthequeen wherearetheavocado readi chvrches arianagrand tonight anyon els see coachella q
lookin samortizphoto
thank god good coffe coachella las menotti sling espresso latt cortado massesand give second wind festiv night
pre festiv vibe amexplatinum hous palm spring inflat art instal friendswithyou acupunctur facial massag eqxbodylab plenti drink bitesand lot fun surpris card member get entri amexplatinum card amexambassador amexlif
good thing americanexpress make life coachella better 

thank much come today part big moment life khunsta0624
dday feel
d2
d3 black feather
soon black feather
season greet
im raffl one boat dismaland wwwchooselov remot control top speed knot batteri includedjust guess much weigh display popup help refuge shop foubert place carnabi street london competit close 22nd decemb
dick pic
free palestin posterb advis poster hand day tourist fair download one websit wwwbanksycouk
open first ever stall trade fair next week ive paint replica separ barrier promot wall hotel iron neighbour complain tall well palestin stand give away free stuff world travel fair london excel centr 6th 7th novemb free get regist onlin today pretend travel agent
shred girl balloon director cut link biosom peopl think didnt realli shred didsom peopl think auction hous werent
urg destroy also creativ urg picasso

none
none
none
matter countri connect young peopl tool resourc attent mayb littl bit inspir best invest make futur young peopl work togeth learn work chang world tha

vamo


dica
protector season april netflix netflixturkiy theprotectornetflix
26042019theprotectortheprotectornetflixnetflixturkiy
theprotector netflixturkiy theprotectornetflix
ell ocak januari 2019elleturkiy
protector netflixtheprotectornetflix netflixturkiy theprotector
theprotector 14122018netflixturkiy theprotectornetflix
en heyecanl süreç balyor hepimizin tüm ekibin elin emein salknow excit part begin huge thank everyon part projecttheprotectornetflixturkiy theprotectornetflix


none
none









dream come true chicama surf gopro

got tri tonal cut edg person digit workout system check stori see full experi still cant get effici tonal electromagnet resist realtim weight adjust tonal offer follow smart accessori link stori
kick festiv season canndesc
blu pledgeworld recent gave uniqu opportun design creat custom boot esquivelsho get feel choos exact piec leather hardwar cut stain shoe learn craftsmen women creat beauti piec everi day see skill patienc creativ goe make shoe left c

call love seat
peac
one imageri ss19 primark nkoang

great ellenvonunwerth ask best frill lizard impress

rip throat
absolut bush
here statement video cf communiti guidelin support view movi safe pleas follow themgoodonwheel jeremiesaund travisflor fight2breath catchingbreath fightlikeabossfilm taylorstroop lungstoryshort cystik1 ashleysros inspir pleas consid donat clairesplacefound cffoundat togeth rais enough awar fund curecfw group cystic fibrosi friend cf transmembran relat disord warrior came togeth excit releas fivefeetapartfilm tomorrow friday march first time cf ever represent plot line major motion pictur incred platform encourag individu old new communiti learn educ other rais fund connect empathi sinc launch five feet apart market campaign cystic fibrosi mention increas hope number grow race toward cure wellso gear tomorrow anticip launch want share way safe moviego unawar cf patient encourag stay six feet apart one anoth found area anoth cf patient may help avoid cross inf

...

justuswest pickupjazzbecom pickup music member pickupmusiccom
ashtonsellar pickupjazzbecom pickup music member pickupmusiccom
tjwhitelaw alway learn guitar tj pickup music master class
wrap film full master class horacebraymus los angel there ton stuff learn includ lick sign advanc exercis music theori becom pickup music member get access drop
may nyc dotanbergman x sophiemarksmus x dotanbergman x julesomus perform next pickup show get tix pickupmusiccomshow igov
weve team dangeliconi anoth round emerg artist fund entri close tonight winner get guitar open pickup show la nyc toward travel cost learn enter read full term pickupmusiccomemergingartist thank curthenderson help us launch
weve film favorit guitarist la jakecurranguitar beaudiakowicz dillanwitherow horacebraymus stay tune pickupjazz

thank everyon came pickup show w mistermcclenney maddiejaymus lilbadsnack










landmark

abchuthai
brown smokey eye use new urbandecaycosmet nakedreload palett liner wick sigmabeauti use mor





















clay court swing timetogrind
good luck adamscottoffici master week watch
excit uniqlo team thank special surpriserf101
101
sad couldnt face anoth epic matchup hope come get better soon rafa fedal
sunshin swing
indian well

thank incred support dubai rf100 atptour
rf100
none
tap flip bat multipl time minut mani time batflip take ceatbatflipchalleng show flip move tap bat flip closer meet dont forget nomin friend
mumbai indian support esa educ sport million live impact caus last year kid rock star
mi boy gear fun food fest esa activ
happi intern pet day
spanish lesson 3month muybien
here big partnership year come adidascricket
race walk make way happi birthday bro kedarjadhavoffici arunkanad
doesnt get picturesqu
ipl folk yuvisoffici hardikpandya93 kieronpollard55 mumbaiindian
behind scene hublot
made us emot special conserv rhino caus close heart sudan meant much us thank olpejeta honour cant believ two month old rhino name africa unbeliev special well bring littl an


burn calori ice cube arrest presid shak theannakais theaktstudio



car wash cuddl shak


gusta ver oprah bailando así love playlist shak repost jznotthatjayz someon tell tim appl chang danc emoji oprah danc emoji oprahmagazin

recordar es vivir papá con sashi rememb live dad sashishak
shoot ryland engag photo
sorri ive gone spiral robe andrew film
life beauti vlogger coupl giveaway
said yes
ryland thank make last year happiest time life thank sleep next even though sweat sheet thank make smile remind love real life even im cancel onlin thank let cover fake tattoo wig video sometim fun saturday night thank love uncondit remind everyday love thank univers everyday bring life cant wait start futur start everi day think lucki love ever even explain word shane
happi blueblood day
love outdoor

sister wive
truth amaz art yashinss
pretti much sum last day jeffre











taghazout youv dream thank much sophiemadtol recommend im pretti sad last day ocean im excit head mountain
feel like ma

black python detail michaellouisni
cant believ im year old today rememb 30th birthday yesterday guess time fli get older 40herewecom birthdaygirl ninaholst
readi weekend


favorit room hous definit livingroom love peac energi swipe see look bought renov amaz paint new furnitur replac fireplac room right beforeandaft førogett oppuss renov stylizimohous arkitekttegnet norwegianhom stue photo nr finnno
love spring weather day soon time get terrac readi love spend time swipe see photo terrac see made lot bigger could enjoy even sun bit longer also get even better view fjord stylizimohouseoutdoor terrass terrasseliv uteplassen hagelivoguterom uterom photo cred finnno
good morn wish love saturday stylizimohous maleri byninaholst byninaholst paint akrylpålerret
look forward spend time terrac stylizimohouseoutdoor terrasseliv terrass hagemøbl uteplassen




tak
upsi




spoko luz


ze spokojem
still absolut favorit video instagram dont know took long download repost feed day one transit said i


life beauti

way back kiew last summer one mani special moment movi come summer look forward hope youll love
good game yesterday special
good reaction last night horribl week
still hurt proud team way play last night head big challeng ahead

heaven look lot better soon ripkarl
good team perform
good team perform



carnav vibesfeat alejoigoa







tag kween
tag kween
tag kween
tag kween
tag kween
tag kween
travel leisur nomin thewebbyaward travel social categori love take littl instagram famili around world us enjoy follow pleas consid vote us link bio
train travel may best way absorb magnific unparallel beauti switzerland mountain countri weve got list scenic swiss train ride link bio hebenj
11thcenturi palac perch feet tyrrhenian sea best amalfi coast view redbett
forget overwat bungalow maldiv resort rent underwat villa dine undersea restaur find link bio tlpick courtesi andredemello
absolut devast today fire icon notr dame cathedr pari heart peopl franc mourn loss wish strength b

In [31]:
count_vectorizer = CountVectorizer(analyzer = "word",tokenizer=None,preprocessor = None,stop_words =None,max_features = 100,ngram_range = (1,2))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20668 entries, 0 to 20667
Data columns (total 7 columns):
hashtags                 8746 non-null object
mentions                 9180 non-null object
location                 6764 non-null object
edge_media_to_caption    20017 non-null object
language                 20668 non-null object
english                  20668 non-null object
tokenStem                20668 non-null object
dtypes: object(7)
memory usage: 1.1+ MB
